Let $\Delta = 30$ days. The VIX is defined as 

$$
    \text{VIX}_T = \sqrt{\frac1\Delta \int_{T}^{T+\Delta} \xi_T^u \, du}
$$

where 

$$
    \xi_T^u = \xi_0^u 
    \exp\left\{ \eta Y_T^u - \eta^2(u^{2H} - (u-T)^{2H})\right\}
$$

and 

$$
    Y_t^u = \sqrt{2H} \int_0^t (u-s)^{H-1/2} \, dW_s, \qquad \forall u \geq t.
$$

We need to compute the covariance matrix of the vector 
$(Y_T^{u_i})_{0 \leq i \leq n}$ where $u_i = T + i \Delta/n$ for $i = 0, 1, \ldots, n$.

Observe that 
$\mathrm{Cov}(Y_T^{u_i}, Y_T^{u_j})=...$